In [2]:
!pip install azure-storage-blob
!pip install pyarrow
!pip install google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO

s1 -> Gather -> Decompresss -> Convert to Tabular -> Clean -> Reformat -> Consolidate -> Transform -> Load

In [4]:
URL = "https://data.cityofnewyork.us/resource/uip8-fykc.csv?$limit=500000"

df_raw = pd.read_csv(URL, engine = 'pyarrow')#, na_values ='(null)')
df_raw.replace('(null)', pd.NA, inplace=True)

print(df_raw.head(10))
print(df_raw.shape)

   arrest_key arrest_date  pd_cd                         pd_desc  ky_cd  \
0   280255493  2024-01-10  397.0  ROBBERY,OPEN AREA UNCLASSIFIED  105.0   
1   279805419  2024-01-02  157.0                          RAPE 1  104.0   
2   279895750  2024-01-03  101.0                       ASSAULT 3  344.0   
3   280809090  2024-01-19  511.0  CONTROLLED SUBSTANCE, POSSESSI  235.0   
4   280357135  2024-01-11  109.0        ASSAULT 2,1,UNCLASSIFIED  106.0   
5   281488967  2024-02-01  157.0                          RAPE 1  104.0   
6   280769124  2024-01-19  464.0                        JOSTLING  230.0   
7   281338081  2024-01-30  101.0                       ASSAULT 3  344.0   
8   280961365  2024-01-23    NaN                            <NA>    NaN   
9   281426006  2024-01-31  155.0                          RAPE 2  104.0   

                      ofns_desc    law_code law_cat_cd arrest_boro  \
0                       ROBBERY  PL 1601001          F           M   
1                          RAPE  P

In [5]:
df_cleaned = df_raw.dropna(axis=0, how='any')
print(df_cleaned.head(10))

    arrest_key arrest_date  pd_cd                         pd_desc  ky_cd  \
0    280255493  2024-01-10  397.0  ROBBERY,OPEN AREA UNCLASSIFIED  105.0   
1    279805419  2024-01-02  157.0                          RAPE 1  104.0   
2    279895750  2024-01-03  101.0                       ASSAULT 3  344.0   
3    280809090  2024-01-19  511.0  CONTROLLED SUBSTANCE, POSSESSI  235.0   
4    280357135  2024-01-11  109.0        ASSAULT 2,1,UNCLASSIFIED  106.0   
5    281488967  2024-02-01  157.0                          RAPE 1  104.0   
6    280769124  2024-01-19  464.0                        JOSTLING  230.0   
7    281338081  2024-01-30  101.0                       ASSAULT 3  344.0   
9    281426006  2024-01-31  155.0                          RAPE 2  104.0   
10   281606761  2024-02-03  105.0               STRANGULATION 1ST  106.0   

                       ofns_desc    law_code law_cat_cd arrest_boro  \
0                        ROBBERY  PL 1601001          F           M   
1                    

In [6]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63612 entries, 0 to 63620
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   arrest_key         63612 non-null  int64         
 1   arrest_date        63612 non-null  datetime64[ns]
 2   pd_cd              63612 non-null  float64       
 3   pd_desc            63612 non-null  object        
 4   ky_cd              63612 non-null  float64       
 5   ofns_desc          63612 non-null  object        
 6   law_code           63612 non-null  object        
 7   law_cat_cd         63612 non-null  object        
 8   arrest_boro        63612 non-null  object        
 9   arrest_precinct    63612 non-null  int64         
 10  jurisdiction_code  63612 non-null  int64         
 11  age_group          63612 non-null  object        
 12  perp_sex           63612 non-null  object        
 13  perp_race          63612 non-null  object        
 14  x_coord_cd 

Use Azure blob for storage

In [8]:
config_file_path = 'config.json'

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'nypdarrest'
blob_name = 'nypdarrest.csv'

output = StringIO()
df_cleaned.to_csv(output, index = False)
data = output.getvalue()
output.close()

blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")



Uploaded nypdarrest.csv to Azure Blob Storage in container nypdarrest.
